In [1]:
import paddle
import torch
paddle.set_device("cpu")
def compare(a, b):
    a = torch.from_numpy(a.numpy()).float()
    b = torch.from_numpy(b.numpy()).float()
    print("mean dif:", (a - b).abs().mean())
    print("max dif:", (a - b).abs().max())

def to_pytorch(x):
    return torch.from_numpy(x.numpy())

def to_paddle(x):
    return paddle.to_tensor(x.numpy())

C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\util\selectors.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import namedtuple, Mapping
C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\_collections.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping
C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\_distutils_hack\__init__.py:19: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


In [2]:
ptx = torch.randn(2,3,4,5,6)
pdx = to_paddle(ptx)

C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 一、paddle.logsumexp不支持5D tensor

In [3]:
o1 = torch.logsumexp(ptx,axis=-1)
o2 = paddle.logsumexp(pdx,axis=-1)

ValueError: (InvalidArgument) The input tensor X's dimensions of logsumexp should be less or equal than 4. But received X's dimensions = 5, X's shape = [2, 3, 4, 5, 6].
  [Hint: Expected x_rank <= 4, but received x_rank:5 > 4:4.] (at C:\home\workspace\Paddle_release2\paddle\fluid\operators\reduce_ops\logsumexp_op.cc:37)
  [operator < logsumexp > error]

In [4]:
# 需要自定义方法，降一个维度。
def _logsumexp(x, axis=-1, keepdim=False):
    if axis < 0:
        axis = x.ndim + axis
    if axis > 1:
        lse = paddle.logsumexp(x.flatten(0, 1), axis=axis - 1, keepdim=keepdim)
        orgshape = x.shape
        if keepdim:
            orgshape[axis] = 1
        else:
            orgshape = orgshape[:axis] + orgshape[axis + 1 :]

        return lse.reshape(shape=orgshape)
    else:
        raise ValueError("axis must greater 1")
o2 = _logsumexp(pdx,axis=-1)
compare(o1,o2)

mean dif: tensor(2.5829e-08)
max dif: tensor(2.3842e-07)


## 二、不支持这种方法增加维度和省略号代表维度


In [5]:
# 不支持这种方法增加维度和省略号代表维度
print(ptx.shape)
print(ptx[:,None,:].shape)
print(ptx[...,-1].shape)
# 需要unsqueeze
print(pdx.unsqueeze([1]).shape)

torch.Size([2, 3, 4, 5, 6])
torch.Size([2, 1, 3, 4, 5, 6])
torch.Size([2, 3, 4, 5])
[2, 1, 3, 4, 5, 6]


In [6]:
# 使用None添加维度，失败
print(pdx[:,None].shape)

ValueError: (InvalidArgument) Currently, VarBase.__getitem__() only allows indexing by Integers, Slices, and tuples of these types, but received NoneType in 2th slice item
  [Hint: Expected PyCheckInteger(slice_item) || ((((PyObject*)(slice_item))->ob_type) == &PySlice_Type) == true, but received PyCheckInteger(slice_item) || ((((PyObject*)(slice_item))->ob_type) == &PySlice_Type):0 != true:1.] (at C:\home\workspace\Paddle_release2\paddle\fluid\pybind\imperative.cc:430)


In [7]:
# 使用省略号，失败
print(pdx[...,-1].shape)

ValueError: (InvalidArgument) Currently, VarBase.__getitem__() only allows indexing by Integers, Slices, and tuples of these types, but received ellipsis in 1th slice item
  [Hint: Expected PyCheckInteger(slice_item) || ((((PyObject*)(slice_item))->ob_type) == &PySlice_Type) == true, but received PyCheckInteger(slice_item) || ((((PyObject*)(slice_item))->ob_type) == &PySlice_Type):0 != true:1.] (at C:\home\workspace\Paddle_release2\paddle\fluid\pybind\imperative.cc:430)


## 三、scatter之间的差异，https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/scatter_cn.html
## 个人找不到好的方法实现pytorch那种的scatter,只好将tensor压平为1D进行scatter（因为文档中说：index （Tensor）- 一维Tensor。 数据类型可以是int32，int64。 index 的长度不能超过 updates 的长度，并且 index 中的值不能超过输入的长度。）

In [8]:
# paddle的代码
def pd_get_sorted_bucket_idx_and_undo_sorted_bucket_idx(
    sequence_length, buckets, num_hashes
):

    def pd_stable_argsort(vector, axis):
        # this function scales the vector so that paddle.argsort is stable.
        # paddle.argsort is not stable on its own
        scale_offset = (
            paddle.arange(vector.shape[axis]).reshape(shape=[1, -1]).astype(vector.dtype)
        )
        scale_offset = scale_offset.expand_as(vector)
        scaled_vector = vector.shape[axis] * vector + (scale_offset % vector.shape[axis])
        return paddle.argsort(scaled_vector, axis=axis)

    # no gradients are needed
    # buckets shape [batch_size, self.num_attention_heads, num_hashes * sequence_length]
    with paddle.no_grad():
        original_shape = buckets.shape
        new_buckets = buckets.flatten(0, 1)
        offsets = (
            paddle.arange(new_buckets.shape[0]) * new_buckets.shape[1]
        ).unsqueeze(-1)
        sorted_bucket_idx = pd_stable_argsort(new_buckets, axis=-1)
        new_sorted_bucket_idx = (sorted_bucket_idx + offsets).flatten()
        updates = paddle.tile(
            paddle.arange(new_buckets.shape[1]), repeat_times=[new_buckets.shape[0]]
        )

        undo_sorted_bucket_idx = paddle.scatter(
            paddle.zeros_like(new_sorted_bucket_idx),
            new_sorted_bucket_idx,
            updates,
            overwrite=True,
        )
        print("=====================================paddle")
        print("sorted_bucket_idx\n",new_sorted_bucket_idx)
        print("indices\n",updates)
    print("undo_sorted_bucket_idx\n",undo_sorted_bucket_idx)
    return sorted_bucket_idx.reshape(
        shape=original_shape
    ), undo_sorted_bucket_idx.reshape(shape=original_shape)

# pytorch的代码
def pt_get_sorted_bucket_idx_and_undo_sorted_bucket_idx(sequence_length, buckets, num_hashes):
    # no gradients are needed
    def pt_stable_argsort(vector, dim):
        # this function scales the vector so that torch.argsort is stable.
        # torch.argsort is not stable on its own
        scale_offset = torch.arange(vector.shape[dim], device=vector.device).view(1, 1, -1)
        scale_offset = scale_offset.expand(vector.shape)
        scaled_vector = vector.shape[dim] * vector + (scale_offset % vector.shape[dim])
        return torch.argsort(scaled_vector, dim=dim)
    with torch.no_grad():
        # hash-based sort
        sorted_bucket_idx = pt_stable_argsort(buckets, dim=-1)

        # create simple indices to scatter to, to have undo sort
        indices = (
            torch.arange(sorted_bucket_idx.shape[-1], device=buckets.device)
            .view(1, 1, -1)
            .expand(sorted_bucket_idx.shape)
        )

        # get undo sort
        undo_sorted_bucket_idx = sorted_bucket_idx.new(*sorted_bucket_idx.size())
        print("=====================================pytorch")
        print("sorted_bucket_idx\n",sorted_bucket_idx)
        print("indices\n",indices)
        undo_sorted_bucket_idx.scatter_(-1, sorted_bucket_idx, indices)

    print("undo_sorted_bucket_idx\n",undo_sorted_bucket_idx)
    return sorted_bucket_idx, undo_sorted_bucket_idx

In [9]:
pt_buckets = torch.randn(2,3,5)
pd_buckets = to_paddle(pt_buckets)

In [10]:
pt_outputs = pt_get_sorted_bucket_idx_and_undo_sorted_bucket_idx(None,pt_buckets,None)
pd_outputs = pd_get_sorted_bucket_idx_and_undo_sorted_bucket_idx(None,pd_buckets,None)

=====================================pytorch
sorted_bucket_idx
 tensor([[[4, 1, 3, 0, 2],
         [0, 2, 3, 1, 4],
         [2, 0, 3, 1, 4]],

        [[4, 0, 1, 2, 3],
         [1, 4, 0, 2, 3],
         [3, 2, 0, 1, 4]]])
indices
 tensor([[[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]],

        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]]])
undo_sorted_bucket_idx
 tensor([[[3, 1, 4, 2, 0],
         [0, 3, 1, 2, 4],
         [1, 3, 0, 2, 4]],

        [[1, 2, 3, 4, 0],
         [2, 0, 3, 4, 1],
         [2, 3, 1, 0, 4]]])
=====================================paddle
sorted_bucket_idx
 Tensor(shape=[30], dtype=int64, place=CPUPlace, stop_gradient=True,
       [4 , 1 , 3 , 0 , 2 , 5 , 7 , 8 , 6 , 9 , 12, 10, 13, 11, 14, 19, 15, 16, 17, 18, 21, 24, 20, 22, 23, 28, 27, 25, 26, 29])
indices
 Tensor(shape=[30], dtype=int64, place=CPUPlace, stop_gradient=True,
       [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 

In [11]:
compare(pt_outputs[0],pd_outputs[0]) # 自定义方法结果一致

mean dif: tensor(0.)
max dif: tensor(0.)


In [12]:
compare(pt_outputs[1],pd_outputs[1]) # 自定义方法结果一致

mean dif: tensor(0.)
max dif: tensor(0.)


## 四、gather区别 https://github.com/PaddlePaddle/X2Paddle/blob/develop/docs/pytorch_project_convertor/API_docs/ops/torch.gather.md
## PaddlePaddle：索引(index)的秩有且只能等于1。

In [13]:
# paddle的代码
def pd_gather_by_expansion(attention_head_size,vectors, idxs, num_hashes):
    expanded_idxs = paddle.tile(
        idxs.unsqueeze(-2), repeat_times=[1, 1, attention_head_size, 1]
    ).reshape(shape=[-1, idxs.shape[2]])
    vectors = (
        paddle.tile(vectors, repeat_times=[1, 1, num_hashes, 1])
        .transpose(perm=[0, 1, 3, 2])
        .reshape(shape=[-1, idxs.shape[2]])
    )

    return (
        paddle.index_sample(vectors, expanded_idxs)
        .reshape(shape=[idxs.shape[0], idxs.shape[1], attention_head_size, -1])
        .transpose(perm=[0, 1, 3, 2])
    )
# pytorch的代码
def pt_gather_by_expansion(attention_head_size, vectors, idxs, num_hashes):
    expanded_idxs = idxs.unsqueeze(-1).expand(-1, -1, -1, attention_head_size)
    vectors = vectors.repeat(1, 1, num_hashes, 1)
    print("expanded_idxs",expanded_idxs)
    return torch.gather(vectors, 2, expanded_idxs)

In [14]:
example = paddle.load("example.pd")

In [15]:
pd_outputs = pd_gather_by_expansion(*example)
pt_outputs = pt_gather_by_expansion(example[0],to_pytorch(example[1]),to_pytorch(example[2]),example[3])

expanded_idxs tensor([[[[  5,   5,   5,  ...,   5,   5,   5],
          [ 70,  70,  70,  ...,  70,  70,  70],
          [ 90,  90,  90,  ...,  90,  90,  90],
          ...,
          [122, 122, 122,  ..., 122, 122, 122],
          [125, 125, 125,  ..., 125, 125, 125],
          [126, 126, 126,  ..., 126, 126, 126]],

         [[  8,   8,   8,  ...,   8,   8,   8],
          [ 57,  57,  57,  ...,  57,  57,  57],
          [124, 124, 124,  ..., 124, 124, 124],
          ...,
          [122, 122, 122,  ..., 122, 122, 122],
          [125, 125, 125,  ..., 125, 125, 125],
          [126, 126, 126,  ..., 126, 126, 126]]],


        [[[  5,   5,   5,  ...,   5,   5,   5],
          [105, 105, 105,  ..., 105, 105, 105],
          [107, 107, 107,  ..., 107, 107, 107],
          ...,
          [124, 124, 124,  ..., 124, 124, 124],
          [125, 125, 125,  ..., 125, 125, 125],
          [126, 126, 126,  ..., 126, 126, 126]],

         [[ 35,  35,  35,  ...,  35,  35,  35],
          [ 34,  34, 

In [16]:
compare(pd_outputs,pt_outputs) # 自定义方法结果一致

mean dif: tensor(0.)
max dif: tensor(0.)


In [17]:
# 想使用paddle.gather 发现失败，因为index必须要为1D
attention_head_size, vectors, idxs, num_hashes = example
expanded_idxs = idxs.unsqueeze(-1).expand(shape=[idxs.shape[0], idxs.shape[1], idxs.shape[2], attention_head_size])
vectors = paddle.tile(vectors, repeat_times=[1, 1, num_hashes, 1]) 
paddle.gather(vectors, expanded_idxs,axis=2) # (InvalidArgument) The index should be 1D, when it is not 2D, but we get 4 

ValueError: (InvalidArgument) The index should be 1D, when it is not 2D, but we get 4
  [Hint: Expected index_dims.size() == 1, but received index_dims.size():4 != 1:1.] (at C:/home/workspace/Paddle_release2/paddle/fluid/operators/gather_op.cc:53)
  [operator < gather > error]